In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import sqlite3
import mysql.connector as cnt
import plotly.express as px

import datetime as dt

# import mysql.connector as cnt
import sqlite3

import requests
from bs4 import BeautifulSoup

# import selenium
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.firefox.options import Options as FirefoxOptions
# Firefox leistų sudėtingesnius nustatymus, bet neveikia per SNAP

from selenium import webdriver
from bs4 import BeautifulSoup
import time

print('Paskaita nr. 46, 2024-10-24')
print('Atverčia ir nuskaito AutoPlius skelbimus pagal anksčiau db išsaugotas nuorodas.')

# https://www.sqlitetutorial.net/sqlite-primary-key/
# https://www.crummy.com/software/BeautifulSoup/bs4/doc/
# https://www.sqlite.org/lang_createtable.html

db_file_name='auto_skelbimai_txt.db'
db_connect = sqlite3.connect(db_file_name)
cs = db_connect.cursor()
print(f"sqlite3.connect('{db_file_name}')")
print(dt.datetime.now().replace(microsecond=0))

Paskaita nr. 46, 2024-10-24
Atverčia ir nuskaito AutoPlius skelbimus pagal anksčiau db išsaugotas nuorodas.
sqlite3.connect('auto_skelbimai_txt.db')
2024-10-24 22:08:28


In [13]:
# https://stackoverflow.com/questions/38076220/python-mysqldb-connection-in-a-class
# https://stackoverflow.com/questions/75688682/how-should-i-type-my-sqlite-connection-and-cursor-in-a-python-class-before-i-pro
# https://pynative.com/python-sqlite/
# https://www.geeksforgeeks.org/python-sqlite-connecting-to-database/
# https://www.reddit.com/r/learnpython/comments/4lznux/best_technique_for_passing_sqlite3_connection/

import sqlite3

class things_with_parameters:
    production=False # 'True' for production, 'False' for testing.

    def __init__(self,name): # constructor.
        self._name=name
        self._connection = sqlite3.connect(name)
        self._cursor = self._connection.cursor()
        things_with_parameters.production or print(f"Connected '{name}'\n")

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.close()
    
    def __str__(self):
        rez=f'The structure of database "{self._name}"\n'
        for (tableName,) in self.query(
        "select NAME from SQLITE_MASTER where TYPE='table' order by NAME;"
        ):
            rez+="{}:\n".format(tableName)
            for (
                columnID, columnName, columnType,
                columnNotNull, columnDefault, columnPK,
            ) in self.query("pragma table_info('{}');".format(tableName)):
                rez+="  {id}: {name}({type}){null}{default}{pk}\n".format(
                    id=columnID,
                    name=columnName,
                    type=columnType,
                    null=" not null" if columnNotNull else "",
                    default=" [{}]".format(columnDefault) if columnDefault else "",
                    pk=" *{}".format(columnPK) if columnPK else "",
                )
        return rez

    @property
    def connection(self):
        return self._connection

    @property
    def cursor(self):
        return self._cursor

    def commit(self):
        self.connection.commit()

    def close(self, commit=True):
        if commit:
            self.commit()
        self.connection.close()
        things_with_parameters.production or print(f"Disconnected '{self._name}'")

    def execute(self, sql, params=None): # result with .fetchall() or .fetchone()
        self._cursor.execute(sql, params or ())

    def fetchall(self):
        return self._cursor.fetchall()

    def fetchone(self):
        return self._cursor.fetchone()

    def query(self, sql, params=None):
        self._cursor.execute(sql, params or ())
        return self.fetchall()

things_with_parameters.production or print("Class 'things_with_parameters' defined.")

Class 'things_with_parameters' defined.


In [16]:
db_objektas=things_with_parameters(db_file_name)
print(db_objektas)
db_objektas.close()

Connected 'auto_skelbimai_txt.db'

The structure of database "auto_skelbimai_txt.db"
object_param:
  0: object_id(INTEGER) not null
  1: param_name(TEXT) not null
  2: param_value(TEXT)
  3: create_timestamp(INTEGER)
  4: update_timestamp(INTEGER)
object_x:
  0: id(INTEGER) *1
  1: href(TEXT)
  2: create_timestamp(INTEGER)
  3: update_timestamp(INTEGER)
param_category:
  0: param_name(TEXT)
  1: param_description(TEXT)
  2: create_timestamp(INTEGER)
  3: update_timestamp(INTEGER)

Disconnected 'auto_skelbimai_txt.db'
